In [152]:
import json
import pathlib

package_paths = {
    "ips": pathlib.Path(".") / "packages" / "hl7.fhir.uv.ips-1.1.0",
    "zib2017": pathlib.Path(".") / "packages" / "nictiz.fhir.nl.stu3.zib2017-2.2.20",
    "zib2020": pathlib.Path(".") / "packages" / "nictiz.fhir.nl.r4.zib2020-0.11.0-beta.1"
}

class Profile:
    def __init__(self, package, profile_name):
        self.package = package
        self.name = profile_name

        content = self._open(self)
        
        # Fish out interesting data: min and max cardinality and binding strengths
        self.of_interest = [el["path"] for el in content["differential"]["element"] if "min" in el or "max" in el or "binding" in el]

    def fits_in(self, other, *ignored):
        self_snap = self._open(self)["snapshot"]["element"]
        other_snap = self._open(other)["snapshot"]["element"]

        self.findings = []
        self.ignored = ignored
        
        # If we want to know if we "fit in" the other, we have to look at the restrictions done by the other and see
        # if we're compatible. We don't need to look at our own restrictions
        for path in other.of_interest:
            try:
                self_el = list(filter(lambda e: e["path"] == path, self_snap))[0]
                other_el = list(filter(lambda e: e["path"] == path, other_snap))[0]
            except IndexError:
                # Happens when we have extensions. Need to look into this
                continue

            if other_el["min"] > self_el["min"]:
                self._finding(path, f"required in {other.name} but optional in {self.name}.")

            if other_el["max"] != "*": # if other is unrestricted we always fit
                if self_el["max"] == "*":
                     self._finding(path, f"restricted to {other_el["max"]} in {other.name} but unrestricted in {self.name}.")
                else:
                    self_max = int(self_el["max"])
                    other_max = int(other_el["max"])
                    if self_max > other_max:
                        self._finding(path, f"restricted to {other_max} in {other.name} but to {self_max} in {self.name}.")

            if "binding" in other_el:
                self_vs = self._getBoundValueSet(self_el)
                other_vs = self._getBoundValueSet(other_el)
                if other_el["binding"]["strength"] == "required":
                    if self_vs != other_vs:
                        self._finding(path, f"required Valueset in {other.name} differs from {self.name}. Manual check needed.", "manual")
                    elif self_el["binding"]["strength"] != "required":
                        self._finding(path, f"required binding in {other.name} is more restrictive than {self_el["binding"]["strength"]} binding in {self.name}.")
                elif other_el["binding"]["strength"] == "extensible":
                    if self_vs != other_vs:
                        self._finding(path, f"required Valueset in {other.name} differs from {self.name}. Manual check needed.", "manual")
                    elif self_el["binding"]["strength"] not in ["extensbile", "required"]:
                        self._finding(path, f"extensible ValueSet in {other.name} is more restrictive than {self_el["binding"]["strength"]} binding in {self.name}.")
                else:
                    if self_vs != other_vs:
                        self._finding(path, f"differences in Valueset preferences. Manual check needed.", "manual")
                    else:
                        # We don't care much about other differences
                        pass

        result = []
        if len(self.findings):
            result.append(f":x: {self.name} ({self.package}) does not seem to fit in {other.name} ({other.package})"),
            for finding in self.findings:
                result.append(f"* {finding}")
        else:
            result.append(f":white_check_mark: {self.name} ({self.package}) does seem to fit in {other.name} ({other.package}).")
        return result

    def _open(self, profile):
        if profile.package == "ips":
            file = package_paths[profile.package] / f"StructureDefinition-{profile.name}.json"
        else:
            file = package_paths[profile.package] / f"{profile.name}.json"

        with open(file) as fd:
            content = json.load(fd)

        return content

    def _getBoundValueSet(self, snapshot_element):
        if "valueSetReference" in snapshot_element["binding"]:
            return snapshot_element["binding"]["valueSetReference"]["reference"]
        return snapshot_element["binding"]["valueSet"]

    def _finding(self, path, message, severity = "error"):
        if path not in self.ignored:
            if severity == "manual":
                symbol = ":interrobang:"
            else:
                symbol = ":x:"
            self.findings.append(f"{symbol} {path}: {message}")
            
zib_Problem2017 = Profile("zib2017", "zib-Problem")
zib_Problem2020 = Profile("zib2020", "zib-Problem")
Condition_uv_ips = Profile("ips", "Condition-uv-ips")
print(zib_Problem2017.fits_in(Condition_uv_ips))
print(Condition_uv_ips.fits_in(zib_Problem2017))
print(zib_Problem2020.fits_in(Condition_uv_ips))
print(Condition_uv_ips.fits_in(zib_Problem2020))

None
None
None
None
